In [77]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F

In [78]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

Converting Crash Data to parquet file

In [4]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/NYC_Vehicle_Collisions_Crashes.parquet"
df.write.parquet(parquet_output_path)


23/08/02 14:35:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/mnt/c/Users/61435/Desktop/Applied Data/Project1/mast30034-project-1-HFISHER00/data/raw/NYC_Vehicle_Collisions_Crashes.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

REMOVING CRASHES WITH NULL FOR BOROUGH (LOCATION)

In [64]:
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("Remove Null Rows").getOrCreate()

# Read the parquet file into a DataFrame
Crash_results = spark.read.parquet('../data/raw/NYC_Vehicle_Collisions_Crashes.parquet')

# Filter out rows with null values in the "BOROUGH" column
Crash_results_filtered = Crash_results.na.drop(subset=["BOROUGH"])

# Show the filtered DataFrame
Crash_results_filtered.show()

# Save the filtered DataFrame to a new Parquet file
Crash_results_filtered.write.parquet('../data/landing/Crash_results_filtered.parquet', mode='overwrite')

+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+------------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME| CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBU

Convert Crime data JSON/csv's to parquets

In [83]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import json

# Read the JSON file into a DataFrame
with open('../data/raw/shooting_data.json', 'r') as f:
    data_list = json.load(f)

df = pd.DataFrame(data_list)

# Convert to Parquet and save the file
table = pa.Table.from_pandas(df)
pq.write_table(table, '../data/raw/shooting_data.parquet')

print("Data converted to Parquet and saved successfully.")


Data converted to Parquet and saved successfully.


In [80]:
from pyspark.sql import SparkSession

# Assuming you already have a SparkSession called "spark"
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")

# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/complaint_data.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/complaint_data.parquet"
df.write.parquet(parquet_output_path)

23/08/04 10:30:01 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/08/04 10:30:05 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:30:05 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/08/04 10:30:21 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:30:22 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/08/04 10:30:23 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:30:24 WARN MemoryManager: Total allocation 

In [82]:
# Step 1: Provide the path to the CSV file
csv_file_path = "../data/raw/arrest_data.csv"

# Step 2: Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Step 3: Write the DataFrame to a Parquet file
parquet_output_path = "../data/raw/arrest_data.parquet"
df.write.parquet(parquet_output_path)

23/08/04 10:32:20 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:32:20 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/08/04 10:32:40 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


Removing unneccesary columns from crime data

Shooting Data

In [89]:
from pyspark.sql import SparkSession

# Step 1: Create a SparkSession
spark = SparkSession.builder.appName("Reduce_Shot_Data").getOrCreate()

# Step 2: Read the existing Parquet file into a DataFrame
parquet_input_path = "../data/raw/shooting_data.parquet"
df = spark.read.parquet(parquet_input_path)

# Step 3: Select the columns you want to keep and rename them
selected_columns = [
    "INCIDENT_KEY as ID",
    "OCCUR_DATE as Date",
    "BORO as Borough",
    "PRECINCT as Precinct",
    "Latitude",
    "Longitude"
]
df_selected = df.selectExpr(*selected_columns)

# Step 4: Write the selected DataFrame to a new Parquet file
parquet_output_path = "../data/raw/shooting_data_reduced.parquet"
df_selected.write.parquet(parquet_output_path)

# Stop the SparkSession
spark.stop()


Complaint Data

In [90]:
from pyspark.sql import SparkSession

# Step 1: Create a SparkSession
spark = SparkSession.builder.appName("Reduce_Complaint_Data").getOrCreate()

# Set the configuration to rebase INT96 datetime values during writing
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY")  # or "CORRECTED" depending on your use case

# Step 2: Read the existing Parquet file into a DataFrame
parquet_input_path = "../data/raw/complaint_data.parquet"
df = spark.read.parquet(parquet_input_path)

# Step 3: Select the columns you want to keep and rename them
selected_columns = [
    "CMPLNT_NUM as ID",
    "CMPLNT_FR_DT as Date",
    "BORO_NM as Borough",
    "ADDR_PCT_CD as Precinct",
    "Latitude",
    "Longitude"
]
df_selected = df.selectExpr(*selected_columns)

# Step 4: Write the selected DataFrame to a new Parquet file
parquet_output_path = "../data/raw/complaint_data_reduced.parquet"
df_selected.write.parquet(parquet_output_path)

df_selected.show()

# Stop the SparkSession
spark.stop()


23/08/04 10:38:23 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:38:23 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/08/04 10:38:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


+---------+----------+-------------+--------+----------------+-----------------+
|       ID|      Date|      Borough|Precinct|        Latitude|        Longitude|
+---------+----------+-------------+--------+----------------+-----------------+
|235489132|10/25/2021|       QUEENS|     104|       40.719592|        -73.87429|
|235662029|10/25/2021|     BROOKLYN|      72|       40.648711|       -74.001185|
|253030475|10/24/2021|    MANHATTAN|      28|       40.804082|       -73.955616|
|235859529|10/29/2021|       QUEENS|     108|       40.742967|       -73.959204|
|235768600|10/28/2021|     BROOKLYN|      73|       40.682875|       -73.910397|
|235393344|10/23/2021|     BROOKLYN|      69|       40.634782|       -73.914405|
|235599134|10/28/2021|     BROOKLYN|      76|       40.680168|       -74.005829|
|235608545|10/27/2021|     BROOKLYN|      84|       40.693462|       -73.987241|
|235557262|10/23/2021|    MANHATTAN|      18|       40.765838|       -73.997454|
|235376644|10/21/2021|     B

Arrest Data

In [91]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Step 1: Create a SparkSession
spark = SparkSession.builder.appName("Reduce_Arrest_Data").getOrCreate()

# Step 2: Read the existing Parquet file into a DataFrame
parquet_input_path = "../data/raw/arrest_data.parquet"
df = spark.read.parquet(parquet_input_path)

# Step 3: Define a mapping dictionary for boroughs
borough_mapping = {
    "B": "Bronx",
    "S": "Staten Island",
    "K": "Brooklyn",
    "M": "Manhattan",
    "Q": "Queens"
}

# Step 4: Select the columns you want to keep and rename them, while mapping borough abbreviations
df_selected = df.select(
    col("ARREST_KEY").alias("ID"),
    col("ARREST_DATE").alias("Date"),
    when(col("ARREST_BORO").isin(list(borough_mapping.keys())), col("ARREST_BORO")).otherwise(col("ARREST_BORO")).alias("Borough"),
    col("ARREST_PRECINCT").alias("Precinct"),
    col("Latitude"),
    col("Longitude")
).replace(borough_mapping, subset=["Borough"])

# Step 5: Write the selected DataFrame to a new Parquet file
parquet_output_path = "../data/raw/arrest_data_reduced.parquet"
df_selected.write.parquet(parquet_output_path)

# Step 6: Show the resulting DataFrame
df_selected.show()

# Step 7: Stop the SparkSession
spark.stop()



23/08/04 10:38:53 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:38:53 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/08/04 10:38:57 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


+---------+----------+---------+--------+------------------+------------------+
|       ID|      Date|  Borough|Precinct|          Latitude|         Longitude|
+---------+----------+---------+--------+------------------+------------------+
|236791704|11/22/2021|Manhattan|      28|40.799008797000056|-73.95240854099995|
|237354740|12/04/2021|    Bronx|      41|40.816391847000034|-73.89529641399997|
|236081433|11/09/2021|   Queens|     113| 40.67970040800003|-73.77604736799998|
| 32311380|06/18/2007|   Queens|      27|              null|              null|
|192799737|01/26/2019|Manhattan|      25|40.800694331000045|-73.94110928599997|
|193260691|02/06/2019|Manhattan|      14| 40.75783900300007|-73.99121211099998|
|237291769|12/03/2021|   Queens|     115| 40.77205649600006|-73.87622400099998|
|236106641|11/10/2021|    Bronx|      41|40.804012949000025|-73.87833183299993|
|238383628|12/28/2021|   Queens|     113| 40.69166001700007|-73.77919852099996|
|149117452|01/06/2016| Brooklyn|      67

Mapping abbreviations of Borough to their full names in Arrest Data 

Merging all crime data

In [92]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Step 1: Create a SparkSession
spark = SparkSession.builder.appName("Merge_Parquet_Files").getOrCreate()

# Set the configuration for legacy mode
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY")

# Step 2: Define the file paths for all the Parquet files you want to merge
file_paths = [
    "../data/raw/arrest_data_reduced.parquet",
    "../data/raw/shooting_data_reduced.parquet",
    "../data/raw/complaint_data_reduced.parquet",
    # Add more file paths as needed
]

# Step 3: Read each Parquet file into a separate DataFrame
dfs = [spark.read.parquet(file_path) for file_path in file_paths]

# Step 4: Add a new column to each DataFrame to identify the source (file name)
source_column_name = "Source"
for i in range(len(dfs)):
    dfs[i] = dfs[i].withColumn(source_column_name, lit(file_paths[i]))

# Step 5: Concatenate or union the DataFrames to create a single DataFrame
merged_df = dfs[0]
for i in range(1, len(dfs)):
    merged_df = merged_df.union(dfs[i])

# Step 6: Write the merged DataFrame to a new Parquet file
parquet_output_path = "../data/raw/merged_crime_data.parquet"
merged_df.write.parquet(parquet_output_path)

merged_df.show()

# Stop the SparkSession
spark.stop()


23/08/04 10:39:13 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:39:13 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/08/04 10:39:18 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:39:21 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:39:21 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/08/04 10:39:24 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/08/04 10:39:25 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014

+---------+----------+---------+--------+------------------+------------------+--------------------+
|       ID|      Date|  Borough|Precinct|          Latitude|         Longitude|              Source|
+---------+----------+---------+--------+------------------+------------------+--------------------+
|236791704|11/22/2021|Manhattan|      28|40.799008797000056|-73.95240854099995|../data/raw/arres...|
|237354740|12/04/2021|    Bronx|      41|40.816391847000034|-73.89529641399997|../data/raw/arres...|
|236081433|11/09/2021|   Queens|     113| 40.67970040800003|-73.77604736799998|../data/raw/arres...|
| 32311380|06/18/2007|   Queens|      27|              null|              null|../data/raw/arres...|
|192799737|01/26/2019|Manhattan|      25|40.800694331000045|-73.94110928599997|../data/raw/arres...|
|193260691|02/06/2019|Manhattan|      14| 40.75783900300007|-73.99121211099998|../data/raw/arres...|
|237291769|12/03/2021|   Queens|     115| 40.77205649600006|-73.87622400099998|../data/raw/

Remove rows with no latitude/ (converting to and from csv as it isnt otherwise working for me)

In [93]:
from pyspark.sql import SparkSession

# Step 1: Create a new SparkSession
with SparkSession.builder.appName("Convert_Parquet_to_CSV").getOrCreate() as spark:
    # Step 2: Read the merged_crime_data.parquet file into a DataFrame
    parquet_input_path = "../data/raw/merged_crime_data.parquet"
    df = spark.read.parquet(parquet_input_path)

    # Step 3: Save the DataFrame as CSV
    csv_output_path = "../data/raw/merged_crime_data.csv"
    df.write.csv(csv_output_path, header=True, mode="overwrite")

In [94]:
import os
import pandas as pd

csv_folder_path = "../data/raw/merged_crime_data.csv"
csv_files = [f for f in os.listdir(csv_folder_path) if f.endswith('.csv')]
for csv_file in csv_files:
    file_path = os.path.join(csv_folder_path, csv_file)
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Drop rows with null latitude and longitude
    df_filtered = df.dropna(subset=['Latitude', 'Longitude'])
    
    # Save the filtered DataFrame back to the CSV file
    df_filtered.to_csv(file_path, index=False)



/tmp/ipykernel_188/2673420837.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_188/2673420837.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_188/2673420837.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [99]:
import os
import pandas as pd
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("Convert_CSV_to_Parquet").getOrCreate()

# Specify the folder path where the CSV files are located
csv_folder_path = "../data/raw/merged_crime_data.csv"

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder_path) if f.endswith('.csv')]

# Iterate through each CSV file, read it into a Pandas DataFrame, convert it to a PySpark DataFrame, and append it to a list of DataFrames
dfs = []
for csv_file in csv_files:
    file_path = os.path.join(csv_folder_path, csv_file)
    
    # Read the CSV file into a Pandas DataFrame
    df_pandas = pd.read_csv(file_path)
    
    # Convert the Pandas DataFrame to a PySpark DataFrame
    df_spark = spark.createDataFrame(df_pandas)
    
    # Append the PySpark DataFrame to the list
    dfs.append(df_spark)

# Concatenate all the DataFrames into a single merged PySpark DataFrame
merged_df_spark = dfs[0]
for df_spark in dfs[1:]:
    merged_df_spark = merged_df_spark.union(df_spark)

# Save the merged PySpark DataFrame to a single Parquet file
parquet_output_path = "../data/raw/merged_crime_data_filtered.parquet"
merged_df_spark.write.parquet(parquet_output_path)

merged_df_spark.show()

# Stop the SparkSession
spark.stop()


ConnectionRefusedError: [Errno 111] Connection refused

Delete all intermediate parquet files

In [68]:
import shutil

# List of folders to delete
folders_to_delete = [
    "../data/raw/arrest_data.parquet",
    "../data/raw/arrest_data_reduced.parquet",
    "../data/raw/complaint_data.parquet",
    "../data/raw/complaint_data_reduced.parquet",
    "../data/raw/shooting_data.parquet",
    "../data/raw/shooting_data_reduced.parquet",
    "../data/raw/merged_crime_data.parquet"
    "../data/raw/filtered_crime_data.parquet"
]

# Function to delete folders and their contents
def delete_folders(folders_list):
    for folder in folders_list:
        try:
            shutil.rmtree(folder)
            print(f"Deleted: {folder}")
        except Exception as e:
            print(f"Error while deleting {folder}: {e}")

# Call the function to delete the specified folders and their contents
delete_folders(folders_to_delete)


Error while deleting ../data/raw/arrest_data.parquet: [Errno 2] No such file or directory: '../data/raw/arrest_data.parquet'
Error while deleting ../data/raw/arrest_data_reduced.parquet: [Errno 2] No such file or directory: '../data/raw/arrest_data_reduced.parquet'
Error while deleting ../data/raw/complaint_data.parquet: [Errno 2] No such file or directory: '../data/raw/complaint_data.parquet'
Error while deleting ../data/raw/complaint_data_reduced.parquet: [Errno 2] No such file or directory: '../data/raw/complaint_data_reduced.parquet'
Error while deleting ../data/raw/shooting_data.parquet: [Errno 2] No such file or directory: '../data/raw/shooting_data.parquet'
Error while deleting ../data/raw/shooting_data_reduced.parquet: [Errno 2] No such file or directory: '../data/raw/shooting_data_reduced.parquet'
Error while deleting ../data/raw/merged_crime_data.parquet../data/raw/filtered_crime_data.parquet: [Errno 2] No such file or directory: '../data/raw/merged_crime_data.parquet../data/